In [1]:
import torch
import torch.optim as optim
import joblib
import optuna
from model.m24odel import MultiDecoderCondVAE
from loss.l24oss_all import integrated_loss_fn

# 1. 환경 및 데이터 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로더 및 스케일러 로드
x_scaler = joblib.load('torch/x_scaler.pkl')
c_saler = joblib.load('torch/c_scaler.pkl')
train_loader = torch.load('torch/train_loader_a_r.pt', weights_only=False)
val_loader = torch.load('torch/val_loader_a_r.pt', weights_only=False)

# 입력 차원 자동 추출 (첫 번째 배치를 통해 확인)
x_sample, c_sample = next(iter(train_loader))
x_dim = x_sample.shape[1]
c_dim = c_sample.shape[1]
z1_dim = 32  # 기존 코드의 기본값 활용

def objective(trial):
    # 2. 튜닝할 하이퍼파라미터 제안
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    z_dim = trial.suggest_int("z_dim", 4, 32)
    alpha = trial.suggest_float("alpha", 0.1, 5.0)
    beta = trial.suggest_float("beta", 0.001, 0.1) # KL 가중치는 보통 작게 시작
    gamma = trial.suggest_float("gamma", 0.1, 10.0)
    surr = trial.suggest_float("surr",0.1,10.0)
    # 3. 모델 및 옵티마이저 선언
    model = MultiDecoderCondVAE(x_dim, c_dim, z_dim, z1_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 각 trial당 학습할 에포크 수 (성능 확인을 위해 최소 20~50회 권장)
    epochs = 50 
    
    for epoch in range(epochs):
        # --- Training Loop ---
        model.train()
        for x, c in train_loader:
            x, c = x.to(device), c.to(device)
            optimizer.zero_grad()
            
            mask_logits, prob_mask, recon_numeric, z_mu, z_logvar,pred_con = model(x, c)
            
            # integrated_loss_fn에 제안된 가중치 적용
            loss_dict = integrated_loss_fn(
                mask_logits, recon_numeric, x, z_mu, z_logvar,pred_con, c
                ,alpha=alpha, beta=beta, gamma=gamma, w_surr = surr
            )
            
            loss_dict['loss'].backward()
            optimizer.step()
        
        # --- Validation Loop ---
        model.eval()
        v_total_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x, v_c = v_x.to(device), v_c.to(device)
                v_mask_logits,_, v_recon_numeric, v_z_mu, v_z_logvar,v_pred = model(v_x, v_c)
                
                v_loss_dict = integrated_loss_fn(
                    v_mask_logits, v_recon_numeric, v_x, v_z_mu, v_z_logvar,v_pred,v_c,
                    alpha=alpha, beta=beta, gamma=gamma,w_surr=surr
                )
                v_total_loss += v_loss_dict['loss'].item()
        
        avg_val_loss = v_total_loss / len(val_loader)
        
        # Pruning: 성능이 개선되지 않는 trial은 조기 종료하여 시간 절약
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

# 4. 최적화 실행
# n_trials: 총 시도 횟수 (예: 30)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# 5. 최적화 결과 확인 및 모델 재학습 활용
print("-" * 30)
print("Best hyperparameters:", study.best_params)
print("Best validation loss:", study.best_value)

# 6. (선택) 파라미터 중요도 시각화
# optuna.visualization.plot_param_importances(study).show()

[I 2026-01-24 22:23:58,964] A new study created in memory with name: no-name-01b8045a-ddb4-4e40-98aa-7e69733f425d
[I 2026-01-24 22:24:28,136] Trial 0 finished with value: 1.5313441063228406 and parameters: {'lr': 0.0001565872486496606, 'z_dim': 17, 'alpha': 3.1430663131668375, 'beta': 0.003461132603689933, 'gamma': 9.15793449868126, 'surr': 9.700462501755151}. Best is trial 0 with value: 1.5313441063228406.
[I 2026-01-24 22:24:56,177] Trial 1 finished with value: 0.07425849767107713 and parameters: {'lr': 0.001963425243783516, 'z_dim': 9, 'alpha': 1.477217570906519, 'beta': 0.08019077589763333, 'gamma': 2.89160769707184, 'surr': 6.8717754405347575}. Best is trial 1 with value: 0.07425849767107713.
[I 2026-01-24 22:25:24,099] Trial 2 finished with value: 2.1627331532930074 and parameters: {'lr': 0.00013018634425749327, 'z_dim': 11, 'alpha': 4.300177074181566, 'beta': 0.03711772901963314, 'gamma': 2.684547756570741, 'surr': 6.025291841917875}. Best is trial 1 with value: 0.07425849767107

------------------------------
Best hyperparameters: {'lr': 0.002340676083161239, 'z_dim': 17, 'alpha': 0.45625233844770874, 'beta': 0.008741819437673534, 'gamma': 7.904571367621319, 'surr': 0.18187154919450677}
Best validation loss: 0.007518944109937078
